In [1]:
"""
- aoi
-- faces
-- identification

- input
-- tracking.csv
-- faces.csv
-- metadata.csv

-----
identification_num = 35 / (번호)_(나이)_(성별)_(정서)
faces_num = 49 / (번호)_(나이)_(성별)_(정서)
"""

'\n- aoi\n-- faces\n-- identification\n\n- input\n-- tracking.csv\n-- faces.csv\n-- metadata.csv\n\n-----\nidentification_num = 35 / (번호)_(나이)_(성별)_(정서)\nfaces_num = 49 / (번호)_(나이)_(성별)_(정서)\n'

In [2]:
# 파일 접근 위한 라이브러리
import os

# 데이터 처리 위한 라이브러리
import pandas as pd
import numpy as np

# px2deg, smoothing 위한 라이브러리
import math
from scipy.signal import savgol_filter

In [168]:
"""
Fixation feature 추출을 위한 함수들입니다.

coordinate_data = dict()
coordinate_data[0:3] = list()
"""

def pass_sav_gol_filter(coordinate_data):
    """
    X와 Y 좌표에 sav_gol filter를 적용하는 함수입니다.
    자세한 내용은 다음 document 참고해주시기 바랍니다.
    5와 3은 제가 가장 적절한 값으로 선택해서 넣어놨는데,
    나중에 데이터 만지시면서 더 적합하다고 판단되는 값 있으면 넣으시면 되겠습니다.
    
    Document = https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.savgol_filter.html
    """
    
    coordinate_data['X'] = savgol_filter(coordinate_data['X'], 5, 3)
    coordinate_data['Y'] = savgol_filter(coordinate_data['Y'], 5, 3)

    return coordinate_data

def get_angular_velocity(coordinate_data, px2deg):
    """
    좌표 데이터를 각속도 데이터로 바꾸는 함수입니다.
    각속도 데이터를 np.array 형태로 return 합니다.
    """
    
    x = coordinate_data['X']
    y = coordinate_data['Y']
    ts = coordinate_data['Timestamp']
    velocity = np.array([np.NaN])

    for i in range(len(x)-1):
        degree = px2deg *  math.sqrt(((x[i+1] - x[i]) ** 2) + ((y[i+1] - y[i]) ** 2))

        velocity = np.append(
            velocity,
            
            (degree / (ts[i+1] - ts[i]) * 1000)
        )

    return velocity

def classify_events(velocity_data, velocity_threshold = 30):
    """
    각속도를 입력 받아서 velocity_threshold를 기준으로 움직임을 분류해 인덱스를 리턴합니다.
    """
    velocity_threshold = velocity_threshold
    # For adaptive velocity_threshold
    # velocity_threshold = find_adaptive_threshold(velocity_data, velocity_threshold)

    saccades = np.where(velocity_data > velocity_threshold)[0]
    fixations = np.where(velocity_data <= velocity_threshold)[0]
    blinks = np.where(np.isnan(velocity_data))[0][1:]

    events = {'saccades': saccades,
              'fixations': fixations,
              'blinks': blinks}

    for event_key, event in events.items():

        start_index = -1
        end_index = -1
        temp_result = []
        result = []
        for i in range(len(event)):
            if i != (len(event) - 1):
                if ((event[i] + 1) == event[i+1]) and (start_index == -1):
                    start_index = i
                elif ((event[i] + 1) == event[i+1]) and (start_index != -1):
                    continue
                elif ((event[i] + 1) != event[i+1]) and (start_index != -1):
                    end_index = i+1
                    temp_result = event[start_index: end_index]
                    result.append(temp_result)
                    start_index = -1
                    end_index = -1
                    temp_result = []
                elif ((event[i] + 1) != event[i+1]) and (start_index == -1):
                    result.append([event[i]])
            else:
                if start_index == -1:
                    result.append([event[i]])
                elif (start_index != -1) and ((event[i] - 1) == event[i - 1]):
                    end_index = i
                    result.append(event[start_index:])

        events[event_key] = result

    return events

# For adaptive velocity threshold
def update_threshold(velocity_data, threshold):
    """
    각속도 역치 값을 찾는 함수에서 사용되는 함수입니다.
    velocity_data의 평균 값과 표준편차를 이용해서 최적의 velocity 값을 찾아갑니다.
    다만, velocity_data의 길이가 짧을 경우(시간 span이 짧을 경우) 제대로 작동하지 않습니다.
    따라서, 충분히 데이터가 모인 뒤에 사용해볼 것을 권장합니다.
    """
    velocity_below_threshold = velocity_data[np.where(velocity_data < threshold)]
    mean = velocity_below_threshold.mean()
    std = velocity_below_threshold.std()
    updated_threshold = mean + (std * 6)

    return updated_threshold

def find_adaptive_threshold(velocity_data, threshold):
    """
    update_threshold 함수를 recursive하게 사용해서 최적의 값을 찾습니다.
    threshold 값을 리턴합니다.
    """
    while abs(update_threshold(velocity_data, threshold) - threshold) > 1:
        threshold = update_threshold(velocity_data, threshold)
    return threshold

def get_classification_result(coordinate_data, velocity_data):
    """
    classify_events() 함수에서 return한 인덱스 값을 input으로 받습니다.
    분류된 인덱스를 이용해 시간을 계산합니다.
    label, start_time, end_time, duration, start_x, start_y, end_x, end_y를 리턴합니다.
    """
    result = classify_events(velocity_data)
    timestamp, x, y = coordinate_data['Timestamp'], coordinate_data['X'], coordinate_data['Y']
    output = []
    for key, value in result.items():
        for i in value:
            start_time = timestamp[i[0]-1]
            end_time = timestamp[i[-1]]
            start_x = x[i[0]-1]
            end_x = x[i[-1]]
            start_y = y[i[0]-1]
            end_y = y[i[-1]]
            output.append([key, start_time, end_time, (end_time-start_time), start_x, start_y, end_x, end_y])
    df_output = pd.DataFrame(data = output,
                             columns = ['label', 'start_time', 'end_time', 'duration', 'start_x', 'start_y', 'end_x', 'end_y'])
    df_output.sort_values(by=['start_time'], axis=0, inplace=True)
    return df_output

In [122]:
def load_input(input_file_name):
    temp_data = pd.read_csv(input_dir + '\\{}'.format(input_file_name))

    return temp_data

def get_identification_fixation(trial_result, item):
    eye_x_aoi_1 = (trial_result["start_x"] >= identification_aoi.at["{}_eye".format(item), 'x1'])
    eye_x_aoi_2 = (trial_result["start_x"] <= identification_aoi.at["{}_eye".format(item), 'x2'])

    eye_x_aoi = (eye_x_aoi_1 & eye_x_aoi_2)

    eye_y_aoi_1 = (trial_result["start_y"] >= identification_aoi.at["{}_eye".format(item), 'y1'])
    eye_y_aoi_2 = (trial_result["start_y"] <= identification_aoi.at["{}_eye".format(item), 'y2'])

    eye_y_aoi = (eye_y_aoi_1 & eye_y_aoi_2)

    mouth_x_aoi_1 = (trial_result["start_x"] >= identification_aoi.at["{}_mouth".format(item), 'x1'])
    mouth_x_aoi_2 = (trial_result["start_x"] <= identification_aoi.at["{}_mouth".format(item), 'x2'])

    mouth_x_aoi = (mouth_x_aoi_1 & mouth_x_aoi_2)

    mouth_y_aoi_1 = (trial_result["start_y"] >= identification_aoi.at["{}_mouth".format(item), 'y1'])
    mouth_y_aoi_2 = (trial_result["start_y"] <= identification_aoi.at["{}_mouth".format(item), 'y2'])

    mouth_y_aoi = (mouth_y_aoi_1 & mouth_y_aoi_2)

    fixation_in_eye_aoi = trial_result[eye_x_aoi & eye_y_aoi]
    fixation_in_mouth_aoi = trial_result[mouth_x_aoi & mouth_y_aoi]

    return fixation_in_eye_aoi, fixation_in_mouth_aoi

def get_faces_fixation(trial_result):
    face11_x_aoi = ((trial_result["start_x"] > faces_aoi.at['faces_11', 'x1']) & (trial_result["start_x"] < faces_aoi.at['faces_11', 'x2']))
    face11_y_aoi = ((trial_result["start_y"] > faces_aoi.at['faces_11', 'y1']) & (trial_result["start_y"] < faces_aoi.at['faces_11', 'y2']))

    face12_x_aoi = ((trial_result["start_x"] > faces_aoi.at['faces_12', 'x1']) & (trial_result["start_x"] < faces_aoi.at['faces_12', 'x2']))
    face12_y_aoi = ((trial_result["start_y"] > faces_aoi.at['faces_12', 'y1']) & (trial_result["start_y"] < faces_aoi.at['faces_12', 'y2']))

    face21_x_aoi = ((trial_result["start_x"] > faces_aoi.at['faces_21', 'x1']) & (trial_result["start_x"] < faces_aoi.at['faces_21', 'x2']))
    face21_y_aoi = ((trial_result["start_y"] > faces_aoi.at['faces_21', 'y1']) & (trial_result["start_y"] < faces_aoi.at['faces_21', 'y2']))

    face22_x_aoi = ((trial_result["start_x"] > faces_aoi.at['faces_22', 'x1']) & (trial_result["start_x"] < faces_aoi.at['faces_22', 'x2']))
    face22_y_aoi = ((trial_result["start_y"] > faces_aoi.at['faces_22', 'y1']) & (trial_result["start_y"] < faces_aoi.at['faces_22', 'y2']))

    total_fixation = trial_result.label.count()
    face11_fixation = trial_result[face11_x_aoi & face11_y_aoi].label.count()
    face12_fixation = trial_result[face12_x_aoi & face12_y_aoi].label.count()
    face21_fixation = trial_result[face21_x_aoi & face21_y_aoi].label.count()
    face22_fixation = trial_result[face22_x_aoi & face22_y_aoi].label.count()

    return total_fixation, face11_fixation, face12_fixation, face21_fixation, face22_fixation

def extract_identification_dwell_time(tracking_data, item):
    trial_duration = max(tracking_data['timestamp'])
    all_point_in_trial = tracking_data.shape[0]    

    eye_x_aoi_1 = (tracking_data["x"] >= identification_aoi.at["{}_eye".format(item), 'x1'])
    eye_x_aoi_2 = (tracking_data["x"] <= identification_aoi.at["{}_eye".format(item), 'x2'])

    eye_x_aoi = eye_x_aoi_1 & eye_x_aoi_2

    eye_y_aoi_1 = (tracking_data["y"] >= identification_aoi.at["{}_eye".format(item), 'y1'])
    eye_y_aoi_2 = (tracking_data["y"] <= identification_aoi.at["{}_eye".format(item), 'y2'])

    eye_y_aoi = eye_y_aoi_1 & eye_y_aoi_2

    mouth_x_aoi_1 = (tracking_data["x"] >= identification_aoi.at["{}_mouth".format(item), 'x1'])
    mouth_x_aoi_2 = (tracking_data["x"] <= identification_aoi.at["{}_mouth".format(item), 'x2'])

    mouth_x_aoi = mouth_x_aoi_1 & mouth_x_aoi_2

    mouth_y_aoi_1 = (tracking_data["y"] >= identification_aoi.at["{}_mouth".format(item), 'y1'])
    mouth_y_aoi_2 = (tracking_data["y"] <= identification_aoi.at["{}_mouth".format(item), 'y2'])

    mouth_y_aoi = mouth_y_aoi_1 & mouth_y_aoi_2

    point_in_eye_aoi = tracking_data[eye_x_aoi & eye_y_aoi]
    point_in_mouth_aoi = tracking_data[mouth_x_aoi & mouth_y_aoi]

    abs_dwell_time_eyes = len(point_in_eye_aoi) / all_point_in_trial * trial_duration
    abs_dwell_time_mouth = len(point_in_mouth_aoi) / all_point_in_trial * trial_duration

    try:
        first_time_eye = tracking_data[eye_x_aoi & eye_y_aoi].iat[0,4]
    except:
        first_time_eye = np.nan

    return abs_dwell_time_eyes, abs_dwell_time_mouth, first_time_eye

def extract_faces_dwelltime(trial_tracking_data):

    trial_duration = max(trial_tracking_data['timestamp'])
    all_point_in_trial = trial_tracking_data.shape[0]   

    face11_x_aoi = (trial_tracking_data["x"] > faces_aoi.at['faces_11', 'x1']) & (trial_tracking_data["x"] < faces_aoi.at['faces_11', 'x2'])
    face11_y_aoi = (trial_tracking_data["y"] > faces_aoi.at['faces_11', 'y1']) & (trial_tracking_data["y"] < faces_aoi.at['faces_11', 'y2'])

    face12_x_aoi = (trial_tracking_data["x"] > faces_aoi.at['faces_12', 'x1']) & (trial_tracking_data["x"] < faces_aoi.at['faces_12', 'x2'])
    face12_y_aoi = (trial_tracking_data["y"] > faces_aoi.at['faces_12', 'y1']) & (trial_tracking_data["y"] < faces_aoi.at['faces_12', 'y2'])

    face21_x_aoi = (trial_tracking_data["x"] > faces_aoi.at['faces_21', 'x1']) & (trial_tracking_data["x"] < faces_aoi.at['faces_21', 'x2'])
    face21_y_aoi = (trial_tracking_data["y"] > faces_aoi.at['faces_21', 'y1']) & (trial_tracking_data["y"] < faces_aoi.at['faces_21', 'y2'])

    face22_x_aoi = (trial_tracking_data["x"] > faces_aoi.at['faces_22', 'x1']) & (trial_tracking_data["x"] < faces_aoi.at['faces_22', 'x2'])
    face22_y_aoi = (trial_tracking_data["y"] > faces_aoi.at['faces_22', 'y1']) & (trial_tracking_data["y"] < faces_aoi.at['faces_22', 'y2'])

    point_in_face11_aoi = trial_tracking_data[face11_x_aoi & face11_y_aoi]
    point_in_face12_aoi = trial_tracking_data[face12_x_aoi & face12_y_aoi]
    point_in_face21_aoi = trial_tracking_data[face21_x_aoi & face21_y_aoi]
    point_in_face22_aoi = trial_tracking_data[face22_x_aoi & face22_y_aoi]

    abs_dwell_time_face11 = len(point_in_face11_aoi) / all_point_in_trial * trial_duration
    abs_dwell_time_face12 = len(point_in_face12_aoi) / all_point_in_trial * trial_duration
    abs_dwell_time_face21 = len(point_in_face21_aoi) / all_point_in_trial * trial_duration
    abs_dwell_time_face22 = len(point_in_face22_aoi) / all_point_in_trial * trial_duration

    return abs_dwell_time_face11, abs_dwell_time_face12, abs_dwell_time_face21, abs_dwell_time_face22

def get_index_of(emotion):
    result = pd.melt(faces_trial_to_item.loc[:, 'face11':'face22']).value.str.split("_").str[-1] == emotion
    return result

In [193]:
institute = 'kirbs'
date = '220623'
pid = '00000000'

def get_result(institute, date, pid):
    
    # 경로 설정 및 파일 불러오기
    current_dir = os.getcwd()
    institute = institute
    date = date
    pid = pid
    identification_trial_to_item = pd.read_csv(current_dir + "\\{}_{}\\input\\identification\\{}_{}_{}_identification.csv".format(institute, date, institute, date, pid))
    faces_trial_to_item = pd.read_csv(current_dir + "\\{}_{}\\input\\faces\\{}_{}_{}_faces.csv".format(institute, date, institute, date, pid), index_col = 'trial')
    meta_data = pd.read_csv(current_dir + "\\{}_{}\\input\\metadata\\{}_{}_{}_metadata.csv".format(institute, date, institute, date, pid))
    tracking_data = pd.read_csv(current_dir + "\\{}_{}\\input\\tracking\\{}_{}_{}_tracking.csv".format(institute, date, institute, date, pid))
    identification_aoi = pd.read_csv(current_dir + "\\{}_{}\\aoi\\identification.csv".format(institute, date), index_col = 'item')
    faces_aoi = pd.read_csv(current_dir + "\\{}_{}\\aoi\\faces.csv".format(institute, date), index_col = 'item')
    
    # px2deg 계산에 필요한 변수를 선언하겠습니다.
    screen_size = meta_data.screen_size[0]
    distance = meta_data.distance[0]
    screen_resolution = meta_data.screen_resolution[0]

    # px2deg 계산
    px2deg = math.degrees(math.atan2(0.5 * screen_size, distance)) / (0.5 * screen_resolution)
    
    # return 파일 선언
    identification_result = pd.DataFrame(columns = [
        'pid',            
        'module',
        'trial',
        'item',
        'total_fixation',
        'eye_fixation',
        'mouth_fixation',
        'abs_dwell_time_eyes',
        'abs_dwell_time_mouth',
        'first_time_eye'])

    faces_result = pd.DataFrame(columns = [
        'pid',
        'module',
        'trial',
        'total_fixation',
        'face11_fixation',
        'face12_fixation',
        'face21_fixation',
        'face22_fixation',
        'total_fixation_under1000',
        'face11_fixation_under1000',
        'face12_fixation_under1000',
        'face21_fixation_under1000',
        'face22_fixation_under1000',
        'abs_dwell_time_face11',
        'abs_dwell_time_face12',
        'abs_dwell_time_face21',
        'abs_dwell_time_face22'])
    
    # identification 데이터 처리 코드입니다.
    identification_tracking_data = tracking_data[tracking_data.module == "identification"]
    for trial in identification_tracking_data.trial.unique():
        item = identification_trial_to_item.at[trial, 'item']
        identification_trial_tracking_data = identification_tracking_data[identification_tracking_data.trial == trial]
        identification_trial_tracking_data = identification_trial_tracking_data.drop_duplicates(['timestamp', 'module', 'trial'], keep = 'last').reset_index().copy()

        ts = np.array(identification_trial_tracking_data.timestamp)
        x = np.array(identification_trial_tracking_data.x)
        y = np.array(identification_trial_tracking_data.y)

        identification_trial_dict = {'Timestamp': ts,
                                    'X' : x,
                                    'Y' : y}

        identification_trial_dict = pass_sav_gol_filter(identification_trial_dict)
        velocity = get_angular_velocity(identification_trial_dict, px2deg)
        identification_trial_result = get_classification_result(identification_trial_dict, velocity)
        identification_trial_result = identification_trial_result[identification_trial_result.label == 'fixations']

        fixation_in_eye_aoi, fixation_in_mouth_aoi = get_identification_fixation(identification_trial_result, item)

        abs_dwell_time_eyes, abs_dwell_time_mouth, first_time_eye = extract_identification_dwell_time(identification_trial_tracking_data, item)

        total_fixation = identification_trial_result.label.count()
        eye_fixation = fixation_in_eye_aoi.label.count()
        mouth_fixation = fixation_in_mouth_aoi.label.count()

        temp_df = pd.DataFrame({
                'pid': pid,
                'module': 'identification',
                'trial': trial,
                'item': item,
                'total_fixation': total_fixation,
                'eye_fixation': eye_fixation,
                'mouth_fixation': mouth_fixation,
                'abs_dwell_time_eyes': abs_dwell_time_eyes,
                'abs_dwell_time_mouth': abs_dwell_time_mouth,
                'first_time_eye': first_time_eye}, index=[0])

        identification_result = pd.concat([identification_result, temp_df])
    identification_result = identification_result.reset_index(drop = True)
    
# faces 데이터 처리 코드입니다.

    faces_tracking_data = tracking_data[tracking_data.module == 'faces']
    for trial in faces_tracking_data.trial.unique():
        # 각 trial의 모든 시간 데이터 분석
        faces_trial_tracking_data = faces_tracking_data[faces_tracking_data.trial == trial]
        faces_trial_tracking_data = faces_trial_tracking_data.drop_duplicates(['timestamp', 'module', 'trial'], keep = 'last').reset_index().copy()

        ts = np.array(faces_trial_tracking_data.timestamp)
        x = np.array(faces_trial_tracking_data.x)
        y = np.array(faces_trial_tracking_data.y)

        faces_trial_dict = {'Timestamp': ts,
                                    'X' : x,
                                    'Y' : y}

        faces_trial_dict = pass_sav_gol_filter(faces_trial_dict)
        velocity = get_angular_velocity(faces_trial_dict, px2deg)
        faces_trial_result = get_classification_result(faces_trial_dict, velocity)
        faces_trial_result = faces_trial_result[faces_trial_result.label == 'fixations']

        # 각 trial의 앞 1초만 데이터 분석

        faces_trial_tracking_data_under1000 = faces_trial_tracking_data[faces_trial_tracking_data.timestamp <= 1000]
        ts_under1000 = np.array(faces_trial_tracking_data_under1000.timestamp)
        x_under1000 = np.array(faces_trial_tracking_data_under1000.x)
        y_under1000 = np.array(faces_trial_tracking_data_under1000.y)

        faces_trial_dict_under1000 = {'Timestamp': ts_under1000,
                                        'X' : x_under1000,
                                        'Y' : y_under1000}

        faces_trial_dict_under1000 = pass_sav_gol_filter(faces_trial_dict_under1000)
        velocity_under1000 = get_angular_velocity(faces_trial_dict_under1000, px2deg)
        faces_trial_result_under1000 = get_classification_result(faces_trial_dict_under1000, velocity_under1000)
        faces_trial_result_under1000 = faces_trial_result_under1000[faces_trial_result_under1000.label == 'fixations']

        # 자극 시간 전체에 걸쳐 AOI 내에 들어온 Fixation 코드
        total_fixation, face11_fixation, face12_fixation, face21_fixation, face22_fixation =\
        get_faces_fixation(faces_trial_result)

        # 자극 시간 전체에 걸쳐 AOI 내에 들어온 dwelltime 코드
        abs_dwell_time_face11, abs_dwell_time_face12, \
        abs_dwell_time_face21, abs_dwell_time_face22 =\
        extract_faces_dwelltime(faces_trial_tracking_data)

        # 첫 1초 동안 AOI 내에 들어온 Fixation 찾는 코드
        total_fixation_under1000, face11_fixation_under1000, \
        face12_fixation_under1000, face21_fixation_under1000, face22_fixation_under1000 =\
        get_faces_fixation(faces_trial_result_under1000)

        temp_df = pd.DataFrame({
                'pid': pid,
                'module': 'faces',
                'trial': trial,
                'total_fixation': total_fixation,
                'face11_fixation': face11_fixation,
                'face12_fixation': face12_fixation,
                'face21_fixation': face21_fixation,
                'face22_fixation': face22_fixation,
                'total_fixation_under1000': total_fixation_under1000,
                'face11_fixation_under1000': face11_fixation_under1000,
                'face12_fixation_under1000': face12_fixation_under1000,
                'face21_fixation_under1000': face21_fixation_under1000,
                'face22_fixation_under1000': face22_fixation_under1000,
                'abs_dwell_time_face11': abs_dwell_time_face11,
                'abs_dwell_time_face12': abs_dwell_time_face12,
                'abs_dwell_time_face21': abs_dwell_time_face21,
                'abs_dwell_time_face22': abs_dwell_time_face22}, index = [0])
        faces_result = pd.concat([faces_result, temp_df])
    faces_result = faces_result.reset_index(drop = True)
    
    # Faces 결과를 각 정서마다의 fixation의 합, dwelltime의 합으로 변환하는 코드입니다.
    
    # 정서 종류를 설정합니다.
    emotions = ['neutral', 'happy', 'sad', 'fearful', 'angry', 'disgusted', 'surprised']

    # 응시자의 각 trial의 정서를 true, false 값으로 변환합니다.
    # emotion_index는 7개의 key를 가지고 len = 49의 value를 가지는 딕셔너리 값입니다.
    emotion_index = {}

    for emotion in emotions:
        emotion_index[emotion] = get_index_of(emotion)

    # dict 자료형인 emotion_index를 dataframe으로 만들어줍시다.
    emotion_index_df = pd.concat(emotion_index, axis=1)

    # emotion_index와 faces_result의 값을 곱할 것입니다.
    # faces_result에서 fixation, fixation_under1000, dwelltime 데이터 프레임을 만들어줍시다.
    fixation_series = pd.melt(faces_result.loc[:, 'face11_fixation': 'face22_fixation']).value
    fixation_df = pd.concat([fixation_series]*7, axis=1)
    fixation_df.columns = ['neutral', 'happy', 'sad', 'fearful', 'angry', 'disgusted', 'surprised']

    fixation_under1000_series = pd.melt(faces_result.loc[:, 'face11_fixation_under1000': 'face22_fixation_under1000']).value
    fixation_under1000_df = pd.concat([fixation_under1000_series]*7, axis=1)
    fixation_under1000_df.columns = ['neutral', 'happy', 'sad', 'fearful', 'angry', 'disgusted', 'surprised']

    dwelltime_series = pd.melt(faces_result.loc[:, 'abs_dwell_time_face11': 'abs_dwell_time_face22']).value
    dwelltime_df = pd.concat([dwelltime_series]*7, axis=1)
    dwelltime_df.columns = ['neutral', 'happy', 'sad', 'fearful', 'angry', 'disgusted', 'surprised']

    # emotion_index와 각 feature별 df를 곱해줍시다.
    emotion_fixation_df = emotion_index_df.mul(fixation_df, axis = 0)
    emotion_fixation_under1000_df = emotion_index_df.mul(fixation_under1000_df, axis = 0)
    emotion_dwelltime_df = emotion_index_df.mul(dwelltime_df, axis = 0)

    # 이제 각 정서별 feature의 합을 구해봅시다.
    faces_emotion_sum_result = {'fixation' : emotion_fixation_df.sum(),
                         'fixation_under1000' : emotion_fixation_under1000_df.sum(),
                         'dwelltime' : emotion_dwelltime_df.sum()}

    # 보기 편하게 행열을 바꿔줍니다.
    faces_emotion_sum_result = pd.DataFrame(faces_emotion_sum_result).transpose()
    
    return identification_result, faces_result, faces_emotion_sum_result

In [ ]:
identification_result, faces_result, emotion_sum_result = get_result(institute, date, pid)

# 이제 각 결과를 excel, csv 등으로 저장해주기만 하면 됩니다.

In [194]:
# 아래는 각 코드 블럭별 결과값 예시입니다.

In [170]:
current_dir = os.getcwd()
institute = institute
date = date
pid = pid
identification_trial_to_item = pd.read_csv(current_dir + "\\{}_{}\\input\\identification\\{}_{}_{}_identification.csv".format(institute, date, institute, date, pid))
faces_trial_to_item = pd.read_csv(current_dir + "\\{}_{}\\input\\faces\\{}_{}_{}_faces.csv".format(institute, date, institute, date, pid), index_col = 'trial')
meta_data = pd.read_csv(current_dir + "\\{}_{}\\input\\metadata\\{}_{}_{}_metadata.csv".format(institute, date, institute, date, pid))
tracking_data = pd.read_csv(current_dir + "\\{}_{}\\input\\tracking\\{}_{}_{}_tracking.csv".format(institute, date, institute, date, pid))
identification_aoi = pd.read_csv(current_dir + "\\{}_{}\\aoi\\identification.csv".format(institute, date), index_col = 'item')
faces_aoi = pd.read_csv(current_dir + "\\{}_{}\\aoi\\faces.csv".format(institute, date), index_col = 'item')

In [177]:
# 불러온 파일을 살펴봅니다.
identification_trial_to_item.head()

# identification에서 각 trial에 해당하는 item이 나와있습니다.
# 이러한 관계쌍은 각 응시자마다 랜덤하게 달라질 것입니다.
# 랜덤하게 달라진 값을 이 파일에 저장하면 되겠습니다.

,trial,item
0,0,1_20_male_happy
1,1,1_20_male_sad
2,2,1_20_male_fearful
3,3,1_20_male_angry
4,4,1_20_male_surprised


In [179]:
faces_trial_to_item.head()

# 이 파일 역시 마찬가지입니다. 관계쌍은 이 파일에 저장하면 됩니다.

,q,face11,face12,face21,face22
trial,,,,,
0,neutral,2_20_male_happy,2_20_male_sad,2_20_male_angry,2_20_male_neutral
1,neutral,3_20_male_happy,3_20_male_sad,3_20_male_angry,3_20_male_neutral
2,neutral,5_20_female_happy,5_20_female_sad,5_20_female_angry,5_20_female_neutral
3,neutral,8_30_male_happy,8_30_male_sad,8_30_male_angry,8_30_male_neutral
4,neutral,11_30_female_happy,11_30_female_sad,11_30_female_angry,11_30_female_neutral


In [181]:
meta_data.head()

# 응시자의 screen_size, distance, screen_resolution 등이 저장되어 있습니다.

,institution,date,screen_size,distance,screen_resolution
0,KIRBS,2022-06-24,62,60,1920


In [183]:
tracking_data.head()

# 응시자의 시선 추적 데이터가 저장되어 있습니다.
# module은 identification과 faces로 구성됩니다.

,module,trial,timestamp,x,y
0,identification,0,0,877.44,236.52
1,identification,0,2,883.20,206.28
2,identification,0,37,881.28,253.80
3,identification,0,69,883.20,328.32
4,identification,0,103,846.72,442.80


In [184]:
identification_aoi.head()

# identification에서 사용된 자극물들의 aoi입니다. eye 먼저 그 다음에 mouth가 위치합니다.
# aoi는 이 파일에서 수정해주면, item에 따라 알아서 찾아서 aoi에 들어온 값들을 계산합니다.

,x1,y1,x2,y2
item,,,,
1_20_male_happy_eye,841,476,1082,581
1_20_male_sad_eye,886,656,1036,761
1_20_male_fearful_eye,841,460,1112,566
1_20_male_angry_eye,871,626,1066,715
1_20_male_surprised_eye,839,475,1082,581


In [185]:
faces_aoi.head()

# faces aoi는 비교적 단순합니다.

,x1,y1,x2,y2
item,,,,
faces_11,583,126,929,504
faces_12,994,126,1334,504
faces_21,583,566,929,943
faces_22,994,566,1334,943


In [171]:
# px2deg 계산에 필요한 변수를 선언하겠습니다.
screen_size = meta_data.screen_size[0]
distance = meta_data.distance[0]
screen_resolution = meta_data.screen_resolution[0]

# px2deg 계산
px2deg = math.degrees(math.atan2(0.5 * screen_size, distance)) / (0.5 * screen_resolution)

# return 파일 선언
identification_result = pd.DataFrame(columns = [
    'pid',            
    'module',
    'trial',
    'item',
    'total_fixation',
    'eye_fixation',
    'mouth_fixation',
    'abs_dwell_time_eyes',
    'abs_dwell_time_mouth',
    'first_time_eye'])

faces_result = pd.DataFrame(columns = [
    'pid',
    'module',
    'trial',
    'total_fixation',
    'face11_fixation',
    'face12_fixation',
    'face21_fixation',
    'face22_fixation',
    'total_fixation_under1000',
    'face11_fixation_under1000',
    'face12_fixation_under1000',
    'face21_fixation_under1000',
    'face22_fixation_under1000',
    'abs_dwell_time_face11',
    'abs_dwell_time_face12',
    'abs_dwell_time_face21',
    'abs_dwell_time_face22'])

In [172]:
# identification 데이터 처리 코드입니다.
identification_tracking_data = tracking_data[tracking_data.module == "identification"]
for trial in identification_tracking_data.trial.unique():
    item = identification_trial_to_item.at[trial, 'item']
    identification_trial_tracking_data = identification_tracking_data[identification_tracking_data.trial == trial]
    identification_trial_tracking_data = identification_trial_tracking_data.drop_duplicates(['timestamp', 'module', 'trial'], keep = 'last').reset_index().copy()

    ts = np.array(identification_trial_tracking_data.timestamp)
    x = np.array(identification_trial_tracking_data.x)
    y = np.array(identification_trial_tracking_data.y)

    identification_trial_dict = {'Timestamp': ts,
                                'X' : x,
                                'Y' : y}

    identification_trial_dict = pass_sav_gol_filter(identification_trial_dict)
    velocity = get_angular_velocity(identification_trial_dict, px2deg)
    identification_trial_result = get_classification_result(identification_trial_dict, velocity)
    identification_trial_result = identification_trial_result[identification_trial_result.label == 'fixations']

    fixation_in_eye_aoi, fixation_in_mouth_aoi = get_identification_fixation(identification_trial_result, item)

    abs_dwell_time_eyes, abs_dwell_time_mouth, first_time_eye = extract_identification_dwell_time(identification_trial_tracking_data, item)

    total_fixation = identification_trial_result.label.count()
    eye_fixation = fixation_in_eye_aoi.label.count()
    mouth_fixation = fixation_in_mouth_aoi.label.count()

    temp_df = pd.DataFrame({
            'pid': pid,
            'module': 'identification',
            'trial': trial,
            'item': item,
            'total_fixation': total_fixation,
            'eye_fixation': eye_fixation,
            'mouth_fixation': mouth_fixation,
            'abs_dwell_time_eyes': abs_dwell_time_eyes,
            'abs_dwell_time_mouth': abs_dwell_time_mouth,
            'first_time_eye': first_time_eye}, index=[0])

    identification_result = pd.concat([identification_result, temp_df])
identification_result = identification_result.reset_index(drop = True)

In [186]:
# identification_result의 데이터는 다음과 같습니다.
identification_result.head()

,pid,module,trial,item,total_fixation,eye_fixation,mouth_fixation,abs_dwell_time_eyes,abs_dwell_time_mouth,first_time_eye
0,00000000,identification,0,1_20_male_happy,1,0,0,2902.292308,0.000000,842.88
1,00000000,identification,1,1_20_male_sad,5,0,1,0.000000,1509.333333,NaN
2,00000000,identification,2,1_20_male_fearful,4,0,0,728.906250,437.343750,925.44
3,00000000,identification,3,1_20_male_angry,8,0,0,36.193798,470.519380,977.28
4,00000000,identification,4,1_20_male_surprised,5,0,1,642.229008,428.152672,1056.00


In [173]:
# faces 데이터 처리 코드입니다.

faces_tracking_data = tracking_data[tracking_data.module == 'faces']
for trial in faces_tracking_data.trial.unique():
    # 각 trial의 모든 시간 데이터 분석
    faces_trial_tracking_data = faces_tracking_data[faces_tracking_data.trial == trial]
    faces_trial_tracking_data = faces_trial_tracking_data.drop_duplicates(['timestamp', 'module', 'trial'], keep = 'last').reset_index().copy()

    ts = np.array(faces_trial_tracking_data.timestamp)
    x = np.array(faces_trial_tracking_data.x)
    y = np.array(faces_trial_tracking_data.y)

    faces_trial_dict = {'Timestamp': ts,
                                'X' : x,
                                'Y' : y}

    faces_trial_dict = pass_sav_gol_filter(faces_trial_dict)
    velocity = get_angular_velocity(faces_trial_dict, px2deg)
    faces_trial_result = get_classification_result(faces_trial_dict, velocity)
    faces_trial_result = faces_trial_result[faces_trial_result.label == 'fixations']

    # 각 trial의 앞 1초만 데이터 분석

    faces_trial_tracking_data_under1000 = faces_trial_tracking_data[faces_trial_tracking_data.timestamp <= 1000]
    ts_under1000 = np.array(faces_trial_tracking_data_under1000.timestamp)
    x_under1000 = np.array(faces_trial_tracking_data_under1000.x)
    y_under1000 = np.array(faces_trial_tracking_data_under1000.y)

    faces_trial_dict_under1000 = {'Timestamp': ts_under1000,
                                    'X' : x_under1000,
                                    'Y' : y_under1000}

    faces_trial_dict_under1000 = pass_sav_gol_filter(faces_trial_dict_under1000)
    velocity_under1000 = get_angular_velocity(faces_trial_dict_under1000, px2deg)
    faces_trial_result_under1000 = get_classification_result(faces_trial_dict_under1000, velocity_under1000)
    faces_trial_result_under1000 = faces_trial_result_under1000[faces_trial_result_under1000.label == 'fixations']

    # 자극 시간 전체에 걸쳐 AOI 내에 들어온 Fixation 코드
    total_fixation, face11_fixation, face12_fixation, face21_fixation, face22_fixation =\
    get_faces_fixation(faces_trial_result)

    # 자극 시간 전체에 걸쳐 AOI 내에 들어온 dwelltime 코드
    abs_dwell_time_face11, abs_dwell_time_face12, \
    abs_dwell_time_face21, abs_dwell_time_face22 =\
    extract_faces_dwelltime(faces_trial_tracking_data)

    # 첫 1초 동안 AOI 내에 들어온 Fixation 찾는 코드
    total_fixation_under1000, face11_fixation_under1000, \
    face12_fixation_under1000, face21_fixation_under1000, face22_fixation_under1000 =\
    get_faces_fixation(faces_trial_result_under1000)

    temp_df = pd.DataFrame({
            'pid': pid,
            'module': 'faces',
            'trial': trial,
            'total_fixation': total_fixation,
            'face11_fixation': face11_fixation,
            'face12_fixation': face12_fixation,
            'face21_fixation': face21_fixation,
            'face22_fixation': face22_fixation,
            'total_fixation_under1000': total_fixation_under1000,
            'face11_fixation_under1000': face11_fixation_under1000,
            'face12_fixation_under1000': face12_fixation_under1000,
            'face21_fixation_under1000': face21_fixation_under1000,
            'face22_fixation_under1000': face22_fixation_under1000,
            'abs_dwell_time_face11': abs_dwell_time_face11,
            'abs_dwell_time_face12': abs_dwell_time_face12,
            'abs_dwell_time_face21': abs_dwell_time_face21,
            'abs_dwell_time_face22': abs_dwell_time_face22}, index = [0])
    faces_result = pd.concat([faces_result, temp_df])
faces_result = faces_result.reset_index(drop = True)

In [187]:
# faces_result의 데이터는 다음과 같습니다.
faces_result.head()

,pid,module,trial,total_fixation,face11_fixation,face12_fixation,face21_fixation,face22_fixation,total_fixation_under1000,face11_fixation_under1000,face12_fixation_under1000,face21_fixation_under1000,face22_fixation_under1000,abs_dwell_time_face11,abs_dwell_time_face12,abs_dwell_time_face21,abs_dwell_time_face22
0,00000000,faces,0,15,6,3,2,0,2,1,0,0,0,1159.000000,729.740741,643.888889,42.925926
1,00000000,faces,1,10,3,0,4,0,3,0,0,1,0,1405.525424,0.000000,2420.627119,39.042373
2,00000000,faces,2,8,2,2,2,0,3,0,1,1,0,1031.777778,884.380952,1068.626984,36.849206
3,00000000,faces,3,13,0,0,3,1,2,0,0,1,0,739.403361,311.327731,933.983193,194.579832
4,00000000,faces,4,13,6,1,2,0,3,1,0,1,0,1353.551724,477.724138,1035.068966,0.000000


In [191]:
# Faces 결과를 각 정서마다의 fixation의 합, dwelltime의 합으로 변환하는 코드입니다.

# 정서 종류를 설정합니다.
emotions = ['neutral', 'happy', 'sad', 'fearful', 'angry', 'disgusted', 'surprised']

# 응시자의 각 trial의 정서를 true, false 값으로 변환합니다.
# emotion_index는 7개의 key를 가지고 len = 49의 value를 가지는 딕셔너리 값입니다.
emotion_index = {}

for emotion in emotions:
    emotion_index[emotion] = get_index_of(emotion)

# dict 자료형인 emotion_index를 dataframe으로 만들어줍시다.
emotion_index_df = pd.concat(emotion_index, axis=1)

# emotion_index와 faces_result의 값을 곱할 것입니다.
# faces_result에서 fixation, fixation_under1000, dwelltime 데이터 프레임을 만들어줍시다.
fixation_series = pd.melt(faces_result.loc[:, 'face11_fixation': 'face22_fixation']).value
fixation_df = pd.concat([fixation_series]*7, axis=1)
fixation_df.columns = ['neutral', 'happy', 'sad', 'fearful', 'angry', 'disgusted', 'surprised']

fixation_under1000_series = pd.melt(faces_result.loc[:, 'face11_fixation_under1000': 'face22_fixation_under1000']).value
fixation_under1000_df = pd.concat([fixation_under1000_series]*7, axis=1)
fixation_under1000_df.columns = ['neutral', 'happy', 'sad', 'fearful', 'angry', 'disgusted', 'surprised']

dwelltime_series = pd.melt(faces_result.loc[:, 'abs_dwell_time_face11': 'abs_dwell_time_face22']).value
dwelltime_df = pd.concat([dwelltime_series]*7, axis=1)
dwelltime_df.columns = ['neutral', 'happy', 'sad', 'fearful', 'angry', 'disgusted', 'surprised']

# emotion_index와 각 feature별 df를 곱해줍시다.
emotion_fixation_df = emotion_index_df.mul(fixation_df, axis = 0)
emotion_fixation_under1000_df = emotion_index_df.mul(fixation_under1000_df, axis = 0)
emotion_dwelltime_df = emotion_index_df.mul(dwelltime_df, axis = 0)

# 이제 각 정서별 feature의 합을 구해봅시다.
faces_emotion_sum_result = {'fixation' : emotion_fixation_df.sum(),
                     'fixation_under1000' : emotion_fixation_under1000_df.sum(),
                     'dwelltime' : emotion_dwelltime_df.sum()}

# 보기 편하게 행열을 바꿔줍니다.
faces_emotion_sum_result = pd.DataFrame(faces_emotion_sum_result).transpose()

In [192]:
# emotion_sum_result의 값은 다음과 같습니다.
faces_emotion_sum_result.head()

,neutral,happy,sad,fearful,angry,disgusted,surprised
fixation,20,132,50,24,124,5,16
fixation_under1000,0,44,10,4,22,0,4
dwelltime,7786.969179,47436.366045,23200.475337,9439.692557,48963.990071,1681.766685,6956.140105
